# Solution to ADM-Homework 2
*by Vianney, Francesco, Onur Ozan and Daniel*

In [ ]:
# if working on colab import
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# importing needed packages
import pandas as pd
import os
from time import perf_counter # out of general interest
import functions
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)


In [5]:
# loading the path to our data from a config-file
data_path = functions.get_from_config("datadir")

## Research questions [RQs]
### [RQ1] 
Exploratory Data Analysis (EDA) - Before working on your research questions, you should provide meaningful statistical summaries through visualizations and tabular tools to understand your data.

### [RQ7] 
Estimating probabilities is a core skill for a data scientist: show us your best!

1. Estimate the probability that a book has over 30% of the ratings above 4.
2. Estimate the probability that an author publishes a new book within two years from its last work.
3. In the file list.json, you will find a peculiar list named "The Worst Books of All Time." Estimate the probability of a book being included in this list, knowing it has more than 700 pages.
4. Are the events X=’Being Included in The Worst Books of All Time list’ and Y=’Having more than 700 pages’ independent? Explain how you have obtained your answer.

#### ad 1.
Estimate the probability that a book has over 30% of the ratings above 4.

After inspecting the .json-file "lighter_authors" we decided to use the attribute "rate_dist".
First we took a look at the format of this attribute and examined its general structure is

'rating_dist': '5:1674064|4:664833|3:231195|2:41699|1:16215|total:2628006'

Just with different numbers, we build our functions to process this information.

In [ ]:
chunksize = 2*10**5
nrows = None
processed_rows = 0
books_with_at_least_one_rating = 0
books_with_30Percent_of_ratings_above_4 = 0

start = perf_counter()
for chunk in pd.read_json( os.path.join( *data_path, "lighter_books_updated" + ".json"), lines=True, chunksize=chunksize, nrows=nrows):
  processed_rows += len(chunk)
  chunk = chunk[["id","ratings_count", "rating_dist"]] # to shrink needed memory
  chunk = chunk[chunk["ratings_count"]>0] # to exclude books without ratings
  books_with_at_least_one_rating += len(chunk)
  chunk["percentage"] = chunk.apply(lambda row : functions.getRatio(functions.parseToDict(row['rating_dist']),['4','5'],'total'), axis=1)
  books_with_30Percent_of_ratings_above_4 += len(chunk[chunk["percentage"] > 0.3]) # we interpret over as strictly over
  print('processed rows:', processed_rows) # in total





processed rows: 199963
processed rows: 399919
processed rows: 599873
processed rows: 799822
processed rows: 999767
processed rows: 1199707
processed rows: 1399605
processed rows: 1599528
processed rows: 1799482
processed rows: 1999430
processed rows: 2199385
processed rows: 2399339
processed rows: 2599297
processed rows: 2799250
processed rows: 2999207
processed rows: 3199162
processed rows: 3399119
processed rows: 3599052
processed rows: 3798957
processed rows: 3998917
processed rows: 4198859
processed rows: 4398801
processed rows: 4598747
processed rows: 4798702
processed rows: 4998653
processed rows: 5198606
processed rows: 5398543
processed rows: 5598492
processed rows: 5798428
processed rows: 5998372
processed rows: 6198316
processed rows: 6398260
processed rows: 6598212
processed rows: 6798158
processed rows: 6998106
processed rows: 7027431


In [ ]:
print('We have found', books_with_30Percent_of_ratings_above_4, 'books with 30 Percent of ratings above 4')
print('We have found', books_with_at_least_one_rating, 'books with at least one rating')
print('The share of books with 30Percent of ratings above 4 is approximately: ', '{:.4f}'.format(books_with_30Percent_of_ratings_above_4 / books_with_at_least_one_rating))
end = perf_counter()
duration = end - start
print('The process lasted approximately', '{:.2f}'.format(duration), "seconds.")


We have found 5661943 books with 30 Percent of ratings above 4
We have found 6017152 books with at least one rating
The share of books with 30Percent of ratings above 4 is approximately:  0.9410
The process lasted approximately 781.74 seconds lasted the process.


### ad 2.
Estimate the probability that an author publishes a new book within two years from its last work.

In [ ]:
# importing further needed packages
from datetime import datetime

In [ ]:
# defining needed functions
def preprocessing(*args, **kwargs):
    """
    Takes a list of strings where each string is representing a date.
    Computes the differences in time of these dates with respect to the first entry of the list using the functions "myStringToDate" and "myDateToSeconds".
    Incompatible strings are dropped.
    Returns None if list-length became 0 during the process. Returns true if the average of the converted list is at most 2 years. Otherwise returns false.
    """
    return_value = None
    res = list(*args, **kwargs)
    # print(res)
    res = [ myStringToDate(entry, '%Y-%m-%d') for entry in res ]
    res = [ entry for entry in res if entry is not None ]
    res = [ myDateToSeconds(res[0], entry) for entry in res]
    res = sorted(set(res)) # first eliminate duplicates, then sort
    return_value = []
    for i in range(len(res)-1):
        return_value.append(res[i+1]-res[i])
    if len(return_value)>0:
        return_value = sum(return_value)/len(return_value) <= 2*60*60*24*365 # returns true if the author is expected to publish a new book within (<=) 2 years
    return return_value

def myStringToDate(date_string, date_format):
    """
    Takes a string representing a date and a string representing a datetime-format.
    Returns the datestring as a datetime-object in the desired format.
    Returns None if the date_string and date_format do not fit.
    """
    res = None
    try:
        res = datetime.strptime(date_string, date_format)
    except ValueError as e:
        print(e)
    return res

def myDateToSeconds(initial_date, current_date):
    """
    Takes an initial date and a current date, both as date-time-objects.
    Returns the total number of seconds from the initial to the current date.
    If an Exception occurs None is returned.
    """
    res = None
    try:
        res = current_date - initial_date
        res = res.total_seconds()
    except Exception as e:
        print(e)
    return res


def listlen(*args,**kwargs):
    return len(list(*args))


In [ ]:
processed_rows = 0
df = None
flag = True
chunksize = 2*10**5
nrows = None # 3*10**5

start = perf_counter()
for chunk in pd.read_json( os.path.join( *data_path, "lighter_books_updated" + ".json"), lines=True, nrows=nrows, chunksize=chunksize):
    processed_rows += len(chunk)
    if flag:
        df = chunk[['author_id','original_publication_date']] # ['author_id','original_publication_date']
        flag = False
    else:
        df = pd.concat([df, chunk[['author_id','original_publication_date']]  ], axis=0)
    print('processed rows:', processed_rows)

df_new = pd.DataFrame(df.groupby('author_id')['original_publication_date'].apply(preprocessing))
df_new

Streaming output truncated to the last 5000 lines.
time data '1996' does not match format '%Y-%m-%d'
time data '1996' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '' does not match format '%Y-%m-%d'
time data '1992' does not match format '%Y-%m-%d'
time data '1991' does not match format '%Y-%m-%d'
time data '2002-11' does not match format '%Y-%m-%d'
time data '1999-06' does not match format '%Y-%m-%d'
time data '1989' does not match format '%Y-%m

,original_publication_date
author_id,
4,True
7,True
10,False
12,False
14,True
...,...
21207692,False
21216155,False
21217422,[]


In [ ]:
results = df_new[ (df_new['original_publication_date'] == True) | (df_new['original_publication_date'] == False)].groupby('original_publication_date')['original_publication_date'].apply(listlen)
# df_new[ df_new['original_publication_date'] == True]
# pd.DataFrame(results)
print( '{:.4f}'.format(results.loc[True]/(results.loc[True]+results.loc[False])), 'is the probability that an author publishes a book within 2 years') # probability
end = perf_counter()
duration = end - start
print('The process lasted approximately', '{:.2f}'.format(duration), "seconds lasted the process.")


0.5201 is the probability that an author publishes a book within 2 years
The process lasted approximately 1651.20 seconds lasted the process.


### ad 3. & 4.
3. In the file list.json, you will find a peculiar list named "The Worst Books of All Time." Estimate the probability of a book being included in this list, knowing it has more than 700 pages.
4. Are the events X=’Being Included in The Worst Books of All Time list’ and Y=’Having more than 700 pages’ independent? Explain how you have obtained your answer.

In [ ]:
# importing further needed packages
import warnings
from scipy.stats import fisher_exact # with this we can perform the fisher test

The fisher test is an excat hypothesis test to check independence of two binary features like given in 4.

In [ ]:
# assessing the books inside the list "The Worst Books of All Time".
df_new = None # for the sake of RAM
list_df = None
chunksize = 10**4
nrows = None # 2*10**4
flag = True

start = perf_counter()
for chunk in pd.read_json(os.path.join( *data_path, 'list' + ".json") , lines=True, chunksize=chunksize, nrows=nrows):
    processed_rows += len(chunk)
    chunk = chunk[chunk['title']=="The Worst Books of All Time"]
    if flag:
        list_df = chunk
        flag = False
    else:
        list_df = pd.concat([list_df, chunk], axis=0)
    print('processed rows:', processed_rows)


processed rows: 7037431
processed rows: 7047431
processed rows: 7057431
processed rows: 7067431
processed rows: 7077431
processed rows: 7087431
processed rows: 7097431
processed rows: 7107431
processed rows: 7108943


In [ ]:
list_df

,id,title,description,description_html,num_pages,num_books,num_voters,created_date,tags,num_likes,created_by,num_comments,books
0,2,The Worst Books of All Time,What do you think are the worst books ever wri...,\n What do you think are the worst books ...,74,7395,18260,"May 20th, 2008","[abominable, abomination, awful, bad, disgusti...",175,"{'name': 'Michael Economy', 'id': '73'}",2570,"[{'book_id': '41865', 'title': 'Twilight', 'au..."


In [ ]:
worst_books_of_all_time = list_df[list_df['title']=="The Worst Books of All Time"]["books"]
if len(worst_books_of_all_time) == 1:
    worst_books_of_all_time = worst_books_of_all_time[0]
else:
    if len(worst_books_of_all_time) < 1:
        warnings.warn("Less than one entry")
        # raise Exception("invalid data")
    if len(worst_books_of_all_time) > 1:
        warnings.warn("More  than 1 entry")

# assessing the book_ids and deleting duplicates
worst_book_ids_of_all_time = [ entry['book_id'] for entry in worst_books_of_all_time ] # assessing the book_ids
if len(worst_book_ids_of_all_time) > len(set(worst_book_ids_of_all_time)):
    worst_book_ids_of_all_time = list(set(worst_book_ids_of_all_time)) # to eliminate duplicates. We checked there are no duplicates.

worst_book_ids_of_all_time

['41865',
 '1162543',
 '49041',
 '428263',
 '10818853',
 '6263078',
 '6076107',
 '3090465',
 '564449',
 '968',
 '4013201',
 '1812457',
 '747061',
 '2183067',
 '48625',
 '993455',
 '569357',
 '3975774',
 '19501',
 '7937462',
 '9742',
 '7455',
 '5107',
 '240469',
 '119322',
 '8714',
 '8752457',
 '11857408',
 '675626',
 '6411961',
 '7613',
 '8135213',
 '9416',
 '3268926',
 '96833',
 '252914',
 '13536860',
 '35220',
 '6867',
 '56495',
 '3636',
 '2657',
 '703847',
 '252917',
 '6487308',
 '13079982',
 '7624',
 '4900',
 '1656001',
 '10920',
 '32261',
 '6478256',
 '70401',
 '801178',
 '139253',
 '43763',
 '662',
 '5308',
 '18144590',
 '227443',
 '37781',
 '629',
 '81176',
 '37442',
 '227603',
 '30183',
 '33926',
 '186190',
 '7604',
 '667',
 '7354',
 '3754102',
 '4989',
 '234225',
 '3332986',
 '168668',
 '5470',
 '5191',
 '10210',
 '11125',
 '6538',
 '930',
 '95707',
 '5641414',
 '78411',
 '5899779',
 '256683',
 '33574273',
 '52529',
 '338798',
 '73968',
 '2956',
 '359770',
 '5129',
 '1035',
 '

In [ ]:
books_with_gt_700_pages_count = 0 # interpreting "more than 700 pages" as "strictly greater than ..."
worst_books_with_gt_700_pages_count = 0

processed_rows = 0
contingency_table = None
flag = True

chunksize = 10**5
nrows = None # 2*10**5


for chunk in pd.read_json(os.path.join( *data_path, "lighter_books_updated" + ".json"), lines=True, chunksize = chunksize, nrows = nrows, dtype=None ):
    processed_rows += len(chunk)
    chunk = chunk[["id","num_pages"]] # to shrink needed memory
    chunk = chunk[chunk["num_pages"]!=""] # to exclude books without pages mentioned.
    chunk.astype({'num_pages':'int'})
    chunk = chunk[chunk["num_pages"]>0] # 0 or less pages is assumed to be a database error or a result of non-knowledge.
    chunk["gt700"] = chunk.apply( lambda row: row['num_pages'] > 700 , axis=1)
    chunk["oneOftheWorst"] = chunk.apply(lambda row : str(row['id']) in worst_book_ids_of_all_time, axis=1) # TODO: str() is necessary. It would be better if both were ints. But this raises errors.
    if flag:
        contingency_table = pd.crosstab(chunk['oneOftheWorst'], chunk['gt700'])
        flag = False
    else:
        contingency_table += pd.crosstab(chunk['oneOftheWorst'], chunk['gt700'])
    print("processed rows:", processed_rows) # in total
    print('current contingency table looks like:\n', contingency_table, "\n\n")

processed rows: 99984
current contingency table looks like:
 gt700          False  True 
oneOftheWorst              
False          69870   3150
True            1882    123 


processed rows: 199963
current contingency table looks like:
 gt700           False  True 
oneOftheWorst               
False          139779   5470
True             2381    139 


processed rows: 299941
current contingency table looks like:
 gt700           False  True 
oneOftheWorst               
False          208006   7436
True             2718    151 


processed rows: 399919
current contingency table looks like:
 gt700           False  True 
oneOftheWorst               
False          275525   9596
True             2939    161 


processed rows: 499897
current contingency table looks like:
 gt700           False  True 
oneOftheWorst               
False          333544  11371
True             3068    164 


processed rows: 599873
current contingency table looks like:
 gt700           False  True 
oneOftheW

<ipython-input-15-0d37ad6a22aa>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["gt700"] = chunk.apply( lambda row: row['num_pages'] > 700 , axis=1)


processed rows: 5198606
current contingency table looks like:
 gt700            False  True 
oneOftheWorst                
False          3050692  91622
True              5284    214 


processed rows: 5298578
current contingency table looks like:
 gt700            False  True 
oneOftheWorst                
False          3071295  92030
True              5295    214 


processed rows: 5398543
current contingency table looks like:
 gt700            False  True 
oneOftheWorst                
False          3089351  92182
True              5299    214 


processed rows: 5498518
current contingency table looks like:
 gt700            False  True 
oneOftheWorst                
False          3143358  93513
True              5307    215 


processed rows: 5598492
current contingency table looks like:
 gt700            False  True 
oneOftheWorst                
False          3194248  94682
True              5325    215 


processed rows: 5698461
current contingency table looks like:
 gt700  

In [ ]:
print("the final contingency table looks like:")
contingency_table

the final contingency table looks like:


gt700,False,True
oneOftheWorst,,
False,3811564,110900
True,5503,220


In [ ]:
print("Now we can provide the answer to 3.")
print("""by computing the ratio of the entry for "gt700 and oneOftheWorst", which is """, contingency_table.loc[True,True], ",")
print("divided by the number of books with over 700 pages, which is", contingency_table.loc[True,False]+contingency_table.loc[True,True] ,".")
print("This yields approximately:", '{:.4f}'.format(contingency_table.loc[True,True] / (contingency_table.loc[True,False]+contingency_table.loc[True,True])))





Now we can provide the answer to 3.
by computing the ratio of the entry for "gt700 and oneOftheWorst", which is  220 ,
divided by the number of books with over 700 pages, which is 5723 .
This yields approximately: 0.0384


In [ ]:
# now performing the fisher test for answering independency
_, pval = fisher_exact(contingency_table)
alpha = 0.05
print('assuming a test-level of alpha =', alpha)
strres = ""
if pval <= alpha:
    strres = ", i.e. less than or equal to alpha, \nthere's significance for dependence."
else:
    strres = ", i.e. strictly greater than alpha, \nthere's no significance for dependence."

print("Since the pvalue is approximately", '{:.6f}'.format(pval), strres)

end = perf_counter()
duration = end - start
print('The process lasted approximately', '{:.2f}'.format(duration), "seconds lasted the process.")


assuming a test-level of alpha = 0.05
Since the pvalue is approximately 0.000011 , i.e. less than or equal to alpha, 
there's significance for dependence.
The process lasted approximately 1911.39 seconds lasted the process.
